# Lab 16: Threat Intelligence Agent

Build an AI agent that gathers and correlates threat intelligence from multiple sources.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/depalmar/ai_for_the_win/blob/main/notebooks/lab16_threat_intel_agent.ipynb)

## Learning Objectives
- Building AI agents with tools
- Threat intelligence data sources
- IOC correlation and enrichment
- MITRE ATT&CK mapping

**Next:** Lab 42 (Security RAG) or Lab 39 (Embeddings Primer)

In [ ]:
# Colab: Install dependencies (skip this cell locally - packages already in venv)
# %pip install -q anthropic

# For Google Colab: Use Secret Manager for secure API key storage
# from google.colab import userdata
# import os
# os.environ['ANTHROPIC_API_KEY'] = userdata.get('ANTHROPIC_API_KEY')

In [ ]:
import json
from typing import List, Dict, Any
from dataclasses import dataclass
from enum import Enum

## 1. Define Tool Schemas

In [ ]:
# Tool definitions for the agent
TOOLS = [
    {
        "name": "lookup_ip",
        "description": "Look up threat intelligence for an IP address",
        "input_schema": {
            "type": "object",
            "properties": {"ip": {"type": "string", "description": "IP address to look up"}},
            "required": ["ip"],
        },
    },
    {
        "name": "lookup_hash",
        "description": "Look up threat intelligence for a file hash",
        "input_schema": {
            "type": "object",
            "properties": {
                "hash": {"type": "string", "description": "File hash (MD5, SHA1, SHA256)"}
            },
            "required": ["hash"],
        },
    },
    {
        "name": "lookup_domain",
        "description": "Look up threat intelligence for a domain",
        "input_schema": {
            "type": "object",
            "properties": {"domain": {"type": "string", "description": "Domain name to look up"}},
            "required": ["domain"],
        },
    },
    {
        "name": "get_mitre_technique",
        "description": "Get details about a MITRE ATT&CK technique",
        "input_schema": {
            "type": "object",
            "properties": {
                "technique_id": {
                    "type": "string",
                    "description": "MITRE technique ID (e.g., T1566)",
                }
            },
            "required": ["technique_id"],
        },
    },
]

## 2. Mock Threat Intel Database

In [ ]:
# Comprehensive Threat Intelligence Database
# Based on real-world threat actor TTPs and IOCs
# NOTE: All malicious IOCs are DEFANGED for safety (use [.] notation)
# Use shared.ioc_utils.refang_ioc() to convert back for analysis tools

THREAT_INTEL_DB = {
    "ips": {
        # APT-related infrastructure - DEFANGED for safety
        "185[.]143[.]223[.]47": {
            "classification": "Malicious",
            "confidence": 95,
            "threat_types": ["C2", "Malware Distribution"],
            "malware_family": "Cobalt Strike",
            "threat_actor": "APT29",
            "country": "Russia",
            "asn": "AS44477",
            "first_seen": "2023-06-15",
            "last_seen": "2024-01-15",
            "related_campaigns": ["SolarWinds", "Cozy Bear Operations"],
            "tags": ["nation-state", "espionage"],
            "related_iocs": ["d41d8cd98f00b204e9800998ecf8427e", "malware-c2[.]evil[.]com"],
        },
        "45[.]33[.]32[.]156": {
            "classification": "Malicious",
            "confidence": 92,
            "threat_types": ["C2", "Ransomware"],
            "malware_family": "LockBit 3.0",
            "threat_actor": "LockBit Group",
            "country": "Netherlands",
            "asn": "AS16276",
            "first_seen": "2023-11-01",
            "last_seen": "2024-01-10",
            "related_campaigns": ["LockBit RaaS"],
            "tags": ["ransomware", "raas", "double-extortion"],
        },
        "203[.]0[.]113[.]50": {
            "classification": "Suspicious",
            "confidence": 70,
            "threat_types": ["Scanning", "Reconnaissance"],
            "country": "China",
            "asn": "AS4134",
            "first_seen": "2024-01-10",
            "last_seen": "2024-01-15",
            "tags": ["scanning", "shodan-like"],
        },
        "91[.]92[.]109[.]23": {
            "classification": "Malicious",
            "confidence": 88,
            "threat_types": ["Phishing", "Credential Theft"],
            "malware_family": "Emotet",
            "threat_actor": "TA542",
            "country": "Ukraine",
            "asn": "AS57043",
            "first_seen": "2023-08-20",
            "last_seen": "2024-01-08",
            "related_campaigns": ["Emotet Resurgence 2023"],
            "tags": ["loader", "banking", "maldoc"],
        },
        "194[.]26[.]29[.]123": {
            "classification": "Malicious",
            "confidence": 91,
            "threat_types": ["C2", "Botnet"],
            "malware_family": "QakBot",
            "threat_actor": "TA570",
            "country": "Russia",
            "asn": "AS49505",
            "first_seen": "2023-09-15",
            "last_seen": "2024-01-12",
            "related_campaigns": ["QakBot Revival"],
            "tags": ["banking", "access-broker"],
        },
        "23[.]106[.]215[.]76": {
            "classification": "Malicious",
            "confidence": 94,
            "threat_types": ["C2", "APT"],
            "malware_family": "PlugX",
            "threat_actor": "APT41",
            "country": "China",
            "asn": "AS398465",
            "first_seen": "2023-05-01",
            "last_seen": "2024-01-14",
            "related_campaigns": ["Double Dragon"],
            "tags": ["nation-state", "espionage", "supply-chain"],
        },
        "159[.]65[.]140[.]11": {
            "classification": "Malicious",
            "confidence": 89,
            "threat_types": ["C2", "Initial Access Broker"],
            "malware_family": "BumbleBee",
            "threat_actor": "Exotic Lily",
            "country": "United States",
            "asn": "AS14061",
            "first_seen": "2023-10-01",
            "last_seen": "2024-01-11",
            "related_campaigns": ["BumbleBee Distribution"],
            "tags": ["loader", "initial-access"],
        },
        "5[.]255[.]100[.]207": {
            "classification": "Malicious",
            "confidence": 96,
            "threat_types": ["C2", "Ransomware"],
            "malware_family": "BlackCat/ALPHV",
            "threat_actor": "BlackCat",
            "country": "Russia",
            "asn": "AS48282",
            "first_seen": "2023-07-15",
            "last_seen": "2024-01-13",
            "related_campaigns": ["ALPHV Triple Extortion"],
            "tags": ["ransomware", "rust", "raas"],
        },
    },
    "hashes": {
        # SHA256 hashes for various malware families
        "d41d8cd98f00b204e9800998ecf8427e": {
            "classification": "Malicious",
            "malware_family": "Emotet",
            "file_type": "PE32 executable",
            "file_name": "invoice_12345.exe",
            "file_size": 245760,
            "first_seen": "2023-11-01",
            "last_seen": "2024-01-08",
            "detection_ratio": "58/72",
            "ssdeep": "6144:rl5XVJKp2I+rJpQe2G6LPms9:rl5lJKp2GJpQe2G6LPm",
            "imphash": "f34d5f2d4577ed6d9ceec516c1f5a744",
            "mitre_techniques": ["T1566.001", "T1059.001", "T1547.001", "T1055"],
            "related_ips": ["91[.]92[.]109[.]23"],
            "tags": ["loader", "banking-trojan", "spam"],
        },
        "a123b456c789d012e345f678g901h234": {
            "classification": "Malicious",
            "malware_family": "Cobalt Strike Beacon",
            "file_type": "PE64 DLL",
            "file_name": "msupdate.dll",
            "file_size": 312832,
            "first_seen": "2023-12-01",
            "last_seen": "2024-01-15",
            "detection_ratio": "45/72",
            "watermark": "1234567890",
            "c2_profile": "jquery-c2.4.6",
            "mitre_techniques": ["T1055.001", "T1071.001", "T1059.001", "T1027"],
            "related_ips": ["185[.]143[.]223[.]47"],
            "tags": ["c2", "red-team-tool", "beacon"],
        },
        "b234c567d890e123f456g789h012i345": {
            "classification": "Malicious",
            "malware_family": "LockBit 3.0",
            "file_type": "PE32 executable",
            "file_name": "locker.exe",
            "file_size": 524288,
            "first_seen": "2023-11-15",
            "last_seen": "2024-01-10",
            "detection_ratio": "62/72",
            "mitre_techniques": ["T1486", "T1490", "T1048", "T1562.001"],
            "related_ips": ["45[.]33[.]32[.]156"],
            "ransomware_config": {
                "extension": ".lockbit",
                "note_name": "README.txt",
                "public_key": "RSA-2048",
            },
            "tags": ["ransomware", "lockbit", "encryption"],
        },
        "c345d678e901f234g567h890i123j456": {
            "classification": "Malicious",
            "malware_family": "QakBot",
            "file_type": "PE32 DLL",
            "file_name": "qbot.dll",
            "file_size": 856064,
            "first_seen": "2023-09-20",
            "last_seen": "2024-01-12",
            "detection_ratio": "52/72",
            "campaign_id": "obama256",
            "mitre_techniques": ["T1055", "T1059.001", "T1547.001", "T1071.001"],
            "related_ips": ["194[.]26[.]29[.]123"],
            "tags": ["banking", "loader", "access-broker"],
        },
        "e567f890g123h456i789j012k345l678": {
            "classification": "Malicious",
            "malware_family": "Mimikatz",
            "file_type": "PE64 executable",
            "file_name": "mim.exe",
            "file_size": 1048576,
            "first_seen": "2023-01-01",
            "last_seen": "2024-01-15",
            "detection_ratio": "68/72",
            "mitre_techniques": ["T1003.001", "T1003.002", "T1003.004", "T1558.003"],
            "tags": ["credential-theft", "red-team-tool", "lsass"],
        },
        "f678g901h234i567j890k123l456m789": {
            "classification": "Malicious",
            "malware_family": "BlackCat/ALPHV",
            "file_type": "PE64 executable (Rust)",
            "file_name": "encrypt.exe",
            "file_size": 2621440,
            "first_seen": "2023-08-01",
            "last_seen": "2024-01-13",
            "detection_ratio": "55/72",
            "mitre_techniques": ["T1486", "T1490", "T1567.002", "T1070.001"],
            "related_ips": ["5[.]255[.]100[.]207"],
            "ransomware_config": {
                "extension": ".alphv",
                "note_name": "RECOVER-FILES.txt",
                "encryption": "ChaCha20+RSA",
            },
            "tags": ["ransomware", "rust", "cross-platform"],
        },
    },
    "domains": {
        "malware-c2[.]evil[.]com": {
            "classification": "Malicious",
            "threat_types": ["C2", "Phishing"],
            "malware_family": "Cobalt Strike",
            "associated_ips": ["185[.]143[.]223[.]47", "185[.]143[.]223[.]48"],
            "registrar": "NameCheap Inc",
            "created": "2023-12-01",
            "expires": "2024-12-01",
            "nameservers": ["ns1[.]suspicous-dns[.]com"],
            "dns_records": {"A": "185[.]143[.]223[.]47", "TXT": "v=spf1 -all"},
            "whois_privacy": True,
            "tags": ["c2", "dga-like"],
        },
        "secure-login-update[.]com": {
            "classification": "Malicious",
            "threat_types": ["Phishing", "Credential Theft"],
            "associated_campaigns": ["Microsoft 365 Phishing"],
            "registrar": "Namecheap Inc",
            "created": "2024-01-05",
            "associated_ips": ["104[.]21[.]45[.]67"],
            "ssl_cert": "Let's Encrypt (newly issued)",
            "mitre_techniques": ["T1566.002", "T1598.003"],
            "tags": ["phishing", "credential-harvesting", "typosquatting"],
        },
        "cdn-microsoft-update[.]net": {
            "classification": "Malicious",
            "threat_types": ["Malware Distribution", "Phishing"],
            "malware_family": "QakBot",
            "associated_ips": ["194[.]26[.]29[.]123"],
            "registrar": "NameSilo",
            "created": "2023-09-10",
            "mitre_techniques": ["T1189", "T1566.002"],
            "tags": ["malware-distribution", "typosquatting"],
        },
        "lockbit-blog[.]onion": {
            "classification": "Malicious",
            "threat_types": ["Ransomware", "Data Leak"],
            "threat_actor": "LockBit Group",
            "network": "Tor",
            "purpose": "Victim shaming site",
            "tags": ["ransomware", "extortion", "darkweb"],
        },
        "update-flash-player[.]xyz": {
            "classification": "Malicious",
            "threat_types": ["Malware Distribution"],
            "malware_family": "SocGholish",
            "associated_ips": ["159[.]65[.]140[.]11"],
            "mitre_techniques": ["T1189", "T1204.002"],
            "tags": ["fake-update", "drive-by"],
        },
    },
    "mitre": {
        "T1566.001": {
            "name": "Spearphishing Attachment",
            "tactic": "Initial Access",
            "description": "Adversaries may send spearphishing emails with a malicious attachment in an attempt to gain access",
            "platforms": ["Windows", "macOS", "Linux"],
            "data_sources": ["Application Log", "Network Traffic", "File Creation"],
            "detection": "Monitor for suspicious email attachments, Office processes spawning cmd/powershell, macro execution events",
            "mitigations": [
                "User Training",
                "Email Filtering",
                "Disable Office Macros",
                "Attachment Sandboxing",
            ],
            "examples": ["Emotet maldoc campaigns", "APT29 phishing operations"],
        },
        "T1059.001": {
            "name": "PowerShell",
            "tactic": "Execution",
            "description": "Adversaries may abuse PowerShell commands and scripts for execution",
            "platforms": ["Windows"],
            "data_sources": [
                "Script Execution",
                "Command: Command Execution",
                "Module: Module Load",
            ],
            "detection": "Enable PowerShell Script Block Logging, monitor for encoded commands (-enc), watch for AMSI bypass attempts",
            "mitigations": [
                "Script Block Logging",
                "Constrained Language Mode",
                "AppLocker",
                "WDAC",
            ],
            "examples": ["Cobalt Strike PowerShell stagers", "Emotet PowerShell downloaders"],
        },
        "T1003.001": {
            "name": "LSASS Memory",
            "tactic": "Credential Access",
            "description": "Adversaries may attempt to access credential material stored in LSASS process memory",
            "platforms": ["Windows"],
            "data_sources": ["Process: Process Access", "Process: OS API Execution"],
            "detection": "Monitor LSASS access (Event 4656/4663), Sysmon Event 10, suspicious DLL loads in LSASS",
            "mitigations": [
                "Credential Guard",
                "LSA Protection",
                "Windows Defender Credential Guard",
            ],
            "examples": ["Mimikatz sekurlsa::logonpasswords", "comsvcs.dll MiniDump", "procdump"],
        },
        "T1486": {
            "name": "Data Encrypted for Impact",
            "tactic": "Impact",
            "description": "Adversaries may encrypt data on target systems to interrupt availability",
            "platforms": ["Windows", "Linux", "macOS"],
            "data_sources": [
                "File: File Modification",
                "Process: Process Creation",
                "Command: Command Execution",
            ],
            "detection": "Monitor for mass file modifications, changes to file extensions, ransom note creation, high entropy changes",
            "mitigations": [
                "Data Backup",
                "Behavior Prevention on Endpoint",
                "Network Segmentation",
            ],
            "examples": ["LockBit", "BlackCat/ALPHV", "Conti", "REvil"],
        },
        "T1490": {
            "name": "Inhibit System Recovery",
            "tactic": "Impact",
            "description": "Adversaries may delete or remove built-in data recovery mechanisms",
            "platforms": ["Windows", "Linux", "macOS"],
            "data_sources": [
                "Command: Command Execution",
                "File: File Deletion",
                "Service: Service Metadata",
            ],
            "detection": "Monitor for vssadmin/wmic shadowcopy commands, bcdedit changes, backup service stops",
            "mitigations": ["Data Backup", "Operating System Configuration"],
            "examples": ["vssadmin delete shadows", "bcdedit /set recoveryenabled No"],
        },
        "T1055": {
            "name": "Process Injection",
            "tactic": "Defense Evasion",
            "description": "Adversaries may inject code into processes to evade defenses and elevate privileges",
            "platforms": ["Windows", "Linux", "macOS"],
            "sub_techniques": ["T1055.001", "T1055.002", "T1055.003", "T1055.004", "T1055.012"],
            "data_sources": [
                "Process: Process Access",
                "Process: Process Modification",
                "Module: Module Load",
            ],
            "detection": "Monitor for suspicious process access, RWX memory regions, unexpected DLL loads",
            "mitigations": ["Behavior Prevention on Endpoint", "Privileged Account Management"],
            "examples": ["Process hollowing", "DLL injection", "Thread hijacking"],
        },
        "T1021.002": {
            "name": "SMB/Windows Admin Shares",
            "tactic": "Lateral Movement",
            "description": "Adversaries may use SMB to access admin shares for lateral movement",
            "platforms": ["Windows"],
            "data_sources": [
                "Network Share: Network Share Access",
                "Logon Session: Logon Session Creation",
            ],
            "detection": "Monitor 5140/5145 events for C$/ADMIN$ access, unusual SMB traffic patterns",
            "mitigations": [
                "Privileged Account Management",
                "Network Segmentation",
                "Disable Admin Shares",
            ],
            "examples": ["PsExec lateral movement", "SMB relay attacks"],
        },
        "T1558.003": {
            "name": "Kerberoasting",
            "tactic": "Credential Access",
            "description": "Adversaries may request service tickets for offline password cracking",
            "platforms": ["Windows"],
            "data_sources": ["Active Directory: Active Directory Credential Request"],
            "detection": "Monitor 4769 events for RC4 encryption type (0x17), multiple TGS requests from single user",
            "mitigations": [
                "Strong Password Policy",
                "Managed Service Accounts",
                "AES Encryption for Service Accounts",
            ],
            "examples": ["Rubeus kerberoast", "Impacket GetUserSPNs.py"],
        },
    },
    "threat_actors": {
        "APT29": {
            "aliases": ["Cozy Bear", "The Dukes", "Nobelium", "Midnight Blizzard"],
            "origin": "Russia",
            "motivation": "Espionage",
            "targets": ["Government", "Think Tanks", "Technology"],
            "ttps": ["T1566.001", "T1059.001", "T1053.005", "T1071.001"],
            "tools": ["Cobalt Strike", "EnvyScout", "BoomBox", "VaporRage"],
            "campaigns": ["SolarWinds", "Microsoft Exchange exploits"],
            "first_seen": "2008",
            "active": True,
        },
        "LockBit Group": {
            "aliases": ["LockBit", "ABCD Ransomware"],
            "origin": "Russia",
            "motivation": "Financial",
            "targets": ["Healthcare", "Manufacturing", "Government"],
            "ttps": ["T1486", "T1490", "T1567.002", "T1048"],
            "tools": ["LockBit ransomware", "StealBit", "Cobalt Strike"],
            "campaigns": ["LockBit 2.0", "LockBit 3.0 (Black)"],
            "first_seen": "2019",
            "active": True,
            "ransomware_model": "RaaS (Ransomware as a Service)",
        },
        "APT41": {
            "aliases": ["Double Dragon", "Winnti Group", "Barium"],
            "origin": "China",
            "motivation": "Espionage and Financial",
            "targets": ["Healthcare", "Gaming", "Telecom", "Technology"],
            "ttps": ["T1190", "T1059.001", "T1055", "T1071.001"],
            "tools": ["PlugX", "ShadowPad", "Winnti", "Cobalt Strike"],
            "campaigns": ["Supply chain attacks", "Vulnerability exploitation"],
            "first_seen": "2012",
            "active": True,
        },
    },
    "vulnerabilities": {
        "CVE-2024-21887": {
            "product": "Ivanti Connect Secure VPN",
            "severity": "CRITICAL",
            "cvss": 9.1,
            "description": "Authentication bypass vulnerability in Ivanti Connect Secure",
            "exploitation": "Active exploitation in the wild",
            "threat_actors": ["UNC5221", "Various nation-state actors"],
            "mitigations": ["Apply vendor patches", "Implement detection rules"],
            "references": ["CISA Advisory", "Mandiant Report"],
        },
        "CVE-2023-44487": {
            "product": "HTTP/2 Protocol",
            "severity": "HIGH",
            "cvss": 7.5,
            "description": "HTTP/2 Rapid Reset Attack (DDoS)",
            "exploitation": "Widespread DDoS attacks observed",
            "mitigations": ["Rate limiting", "Web application firewall rules"],
            "affected": ["Nginx", "Apache", "Various CDNs"],
        },
    },
}

# Print database statistics
print("=" * 60)
print("COMPREHENSIVE THREAT INTELLIGENCE DATABASE")
print("=" * 60)
print(f"\n  Database Statistics:")
print(f"   Malicious IPs: {len(THREAT_INTEL_DB['ips'])}")
print(f"   File Hashes: {len(THREAT_INTEL_DB['hashes'])}")
print(f"   Malicious Domains: {len(THREAT_INTEL_DB['domains'])}")
print(f"   MITRE Techniques: {len(THREAT_INTEL_DB['mitre'])}")
print(f"   Threat Actors: {len(THREAT_INTEL_DB['threat_actors'])}")
print(f"   Vulnerabilities: {len(THREAT_INTEL_DB['vulnerabilities'])}")

print("\n  NOTE: All IOCs are DEFANGED for safety (use [.] notation)")
print("  Use shared.ioc_utils.refang_ioc() to convert for analysis tools")

print("\n  Threat Actor Coverage:")
for actor, info in THREAT_INTEL_DB["threat_actors"].items():
    print(f"   {actor} ({info['origin']}): {info['motivation']}")

print("\n  Malware Families Covered:")
families = set()
for ip in THREAT_INTEL_DB["ips"].values():
    if "malware_family" in ip:
        families.add(ip["malware_family"])
for h in THREAT_INTEL_DB["hashes"].values():
    if "malware_family" in h:
        families.add(h["malware_family"])
for f in sorted(families):
    print(f"   - {f}")

## 3. Tool Implementation

In [ ]:
def lookup_ip(ip: str) -> Dict:
    """Look up IP in threat intel database."""
    if ip in THREAT_INTEL_DB["ips"]:
        return THREAT_INTEL_DB["ips"][ip]
    return {"classification": "Unknown", "message": "No threat data found"}


def lookup_hash(hash_value: str) -> Dict:
    """Look up file hash in threat intel database."""
    if hash_value in THREAT_INTEL_DB["hashes"]:
        return THREAT_INTEL_DB["hashes"][hash_value]
    return {"classification": "Unknown", "message": "No threat data found"}


def lookup_domain(domain: str) -> Dict:
    """Look up domain in threat intel database."""
    if domain in THREAT_INTEL_DB["domains"]:
        return THREAT_INTEL_DB["domains"][domain]
    return {"classification": "Unknown", "message": "No threat data found"}


def get_mitre_technique(technique_id: str) -> Dict:
    """Get MITRE ATT&CK technique details."""
    if technique_id in THREAT_INTEL_DB["mitre"]:
        return THREAT_INTEL_DB["mitre"][technique_id]
    return {"error": f"Technique {technique_id} not found"}


def execute_tool(name: str, args: Dict) -> str:
    """Execute a tool and return result."""
    if name == "lookup_ip":
        result = lookup_ip(args["ip"])
    elif name == "lookup_hash":
        result = lookup_hash(args["hash"])
    elif name == "lookup_domain":
        result = lookup_domain(args["domain"])
    elif name == "get_mitre_technique":
        result = get_mitre_technique(args["technique_id"])
    else:
        result = {"error": f"Unknown tool: {name}"}

    return json.dumps(result, indent=2)

## 4. Test Tools

## Interpreting Threat Intelligence Lookups

### Understanding Confidence Scores

| Score Range | Meaning | Action |
|-------------|---------|--------|
| **90-100%** | High confidence malicious | Block immediately, investigate |
| **70-89%** | Likely malicious | Block, monitor for confirmation |
| **50-69%** | Suspicious | Monitor closely, gather more intel |
| **< 50%** | Low confidence | Don't block, but log |

### Key Fields in IP Intelligence

- **Classification**: Malicious/Suspicious/Clean - the verdict
- **Threat Types**: What the IP is used for (C2, Phishing, Scanning)
- **Malware Family**: Specific malware using this infrastructure
- **Threat Actor**: Attribution (if known) - nation-state vs criminal
- **First/Last Seen**: How fresh is this IOC? Old IOCs may be abandoned

### Key Fields in Hash Intelligence

- **Detection Ratio**: "58/72" means 58 of 72 AV engines detect it
  - High ratio (>50/72) = well-known malware
  - Low ratio (<20/72) = possibly new/evasive
- **MITRE Techniques**: What the malware can do (crucial for impact assessment)
- **Related IOCs**: Other indicators to hunt for

### Key Fields in Domain Intelligence

- **Registration Date**: Recent registration = higher suspicion
- **WHOIS Privacy**: Legitimate businesses usually don't hide WHOIS
- **SSL Certificate**: Let's Encrypt with very recent issuance = suspicious
- **Typosquatting**: Domains mimicking legitimate services

In [ ]:
# Test the tools with DEFANGED IOCs
print("IP Lookup (defanged for safety):")
print(execute_tool("lookup_ip", {"ip": "185[.]143[.]223[.]47"}))

print("\nDomain Lookup (defanged for safety):")
print(execute_tool("lookup_domain", {"domain": "malware-c2[.]evil[.]com"}))

print("\nMITRE Technique:")
print(execute_tool("get_mitre_technique", {"technique_id": "T1566.001"}))

## 5. Agent Loop

In [ ]:
class ThreatIntelAgent:
    """AI agent for threat intelligence gathering.

    Uses real AI when API key is available, otherwise provides a mock response
    that clearly indicates it's not using actual AI capabilities.
    """

    def __init__(self):
        try:
            from anthropic import Anthropic

            self.client = Anthropic()
            self.available = True
            print("  AI Agent initialized with Anthropic API")
        except:
            self.available = False
            print("  AI not available - using mock responses")
            print("  Set ANTHROPIC_API_KEY for real AI analysis")

    def investigate(self, query: str) -> str:
        """Investigate a threat using available tools.

        When AI is available, the agent will:
        - Analyze the query to determine which tools to use
        - Execute multiple tool calls autonomously
        - Synthesize findings into a comprehensive report
        - Provide actionable recommendations

        Token limit set to 4096 for detailed analysis.
        """
        if not self.available:
            return self._mock_investigation(query)

        messages = [{"role": "user", "content": query}]

        # Agent loop - the AI will autonomously call tools as needed
        while True:
            response = self.client.messages.create(
                model="claude-sonnet-4.5",
                max_tokens=4096,  # Increased for detailed analysis
                tools=TOOLS,
                messages=messages,
            )

            # Check if we need to call tools
            if response.stop_reason == "tool_use":
                # Execute tools
                tool_results = []
                for content in response.content:
                    if content.type == "tool_use":
                        print(f"   Tool call: {content.name}({content.input})")
                        result = execute_tool(content.name, content.input)
                        tool_results.append(
                            {"type": "tool_result", "tool_use_id": content.id, "content": result}
                        )

                # Add to messages
                messages.append({"role": "assistant", "content": response.content})
                messages.append({"role": "user", "content": tool_results})
            else:
                # Final response - this is real AI-generated analysis
                return response.content[0].text

    def _mock_investigation(self, query: str) -> str:
        """Mock response when AI is not available.

        NOTE: This is a STATIC response, not AI-generated.
        For real AI analysis, set your ANTHROPIC_API_KEY.
        """
        return """
==============================================================================
                         MOCK RESPONSE (NO AI)
==============================================================================
  This is a STATIC mock response because no API key is configured.
  To see real AI-powered threat intelligence analysis:

  1. Get an API key from https://console.anthropic.com/
  2. Set it in Colab: Secrets -> Add ANTHROPIC_API_KEY
  3. Re-run the notebook

==============================================================================
                    EXAMPLE THREAT INTELLIGENCE REPORT
==============================================================================

### Investigation Summary
Based on the IOCs provided, here's what a real AI analysis would include:

### IP Analysis: 185[.]143[.]223[.]47
- **Classification**: Malicious (95% confidence)
- **Threat Types**: C2 Server, Malware Distribution
- **Malware Family**: Cobalt Strike
- **Associated Campaigns**: APT29, Cozy Bear
- **Location**: Russia

### Related MITRE ATT&CK Techniques
- T1566.001: Spearphishing Attachment (Initial Access)
- T1059.001: PowerShell (Execution)
- T1055: Process Injection (Defense Evasion)

### Threat Actor Profile
- **Name**: APT29 (Cozy Bear)
- **Motivation**: Nation-state espionage
- **Targets**: Government, Technology, Think Tanks

### Recommendations
1. Block IP at perimeter firewall
2. Search for this IP in historical logs
3. Enable PowerShell script block logging
4. Alert on Cobalt Strike beacon patterns
5. Review network traffic for C2 beaconing patterns

==============================================================================
  NOTE: Real AI would provide dynamic analysis based on your specific query
  and correlate across multiple intelligence sources in real-time.
==============================================================================
"""

## Understanding Agent Behavior

### The Agent Loop Explained

```
┌──────────────────────────────────────────────────────────┐
│                     AGENT LOOP                           │
├──────────────────────────────────────────────────────────┤
│                                                          │
│   User Query: "Investigate IP 185.143.223.47"           │
│         │                                                │
│         ▼                                                │
│   ┌─────────────┐                                        │
│   │    LLM      │ ← Decides which tools to call         │
│   └──────┬──────┘                                        │
│          │                                               │
│          ▼                                               │
│   Tool Call: lookup_ip("185.143.223.47")                │
│          │                                               │
│          ▼                                               │
│   Tool Result: {classification: "Malicious", ...}       │
│          │                                               │
│          ▼                                               │
│   ┌─────────────┐                                        │
│   │    LLM      │ ← Decides: need more tools?           │
│   └──────┬──────┘                                        │
│          │                                               │
│     ┌────┴────┐                                          │
│     │         │                                          │
│    YES        NO                                         │
│     │         │                                          │
│     ▼         ▼                                          │
│   More      Final                                        │
│   Tools     Response                                     │
│                                                          │
└──────────────────────────────────────────────────────────┘
```

### Why Multi-Step Reasoning Matters

Single query → Multiple tool calls → Correlated intelligence

Example chain for "Investigate IP 185.143.223.47":
1. `lookup_ip` → Returns "Cobalt Strike" malware family
2. `get_mitre_technique("T1071.001")` → Learns about C2 communication
3. `lookup_domain` → Checks associated domains
4. **Final synthesis**: Comprehensive threat report

### Reading the Agent's "Thinking"

When you see output like:
```
Tool call: lookup_ip({'ip': '185.143.223.47'})
Tool call: get_mitre_technique({'technique_id': 'T1566.001'})
```

This shows the agent's autonomous decision-making:
- It chose to look up the IP first
- Based on results, it decided to get MITRE technique details
- Each call builds on previous information

In [ ]:
# Run investigation with defanged IOC
# Using IPython display for proper markdown rendering in Colab
from IPython.display import display, Markdown

agent = ThreatIntelAgent()
result = agent.investigate(
    "Investigate IP 185[.]143[.]223[.]47 and provide threat intelligence with MITRE mappings"
)

# Render the output as formatted markdown
display(Markdown(result))

## Summary

In this lab, we built a threat intelligence agent with:

1. **Tool Definitions** - Structured schemas for API-like tools
2. **Mock Database** - Simulated threat intel data
3. **Agent Loop** - Autonomous tool execution
4. **MITRE Mapping** - Technique correlation

### Key Takeaways:
- Agents can chain multiple tool calls
- Tool definitions guide LLM behavior
- MITRE ATT&CK provides common language
- Correlation across sources adds value

### Next Steps:
1. Add real threat intel APIs (VirusTotal, Shodan)
2. Implement caching for repeated lookups
3. Build automated IOC enrichment pipeline